# Mediawiki content history dumps

https://wikitech.wikimedia.org/wiki/Data_Platform/Data_Lake/Content/Mediawiki_content_history_v1


## Starting spark session

In [ ]:
from pyspark.sql import functions as F, types as T, Window
import wmfdata
spark = wmfdata.spark.create_session(
    app_name='Pyspark notebook MGerlach_(WMF)', 
    # ship_python_env=True,
)
spark

# Read table

In [ ]:
df = (
    spark.read.table("wmf_content.mediawiki_content_history_v1")
    # only content namespace
    .where(F.col("page_namespace_id")==0)
    # only English Wikipedia
    .where(F.col("wiki_id")=="enwiki")
    # get wikitext as separate column
    .withColumn("wikitext", F.col("revision_content_slots").getItem("main").getField("content_body"))
    # filter out redirect pages, currently broken
    # see https://phabricator.wikimedia.org/T400632
    .filter(~F.col("wikitext").contains("#REDIRECT"))
    .filter(
        F.col("page_redirect_target").isNull() | (F.col("page_redirect_target") == "")
    )
)

df.show()